# 言語理解（ルールによる方法）
ここでは、言語理解を実装します。まずはルールによる方法で行います。扱う方法は、意味文法と意味・格フレームです。

## 事前の設定
- MeCabのインストール
- MeCabをpythonで使用するためのmecab-pythonのインストール

In [1]:
# 必要なラブラリを読み込む
import re

In [2]:
# 入力データを読み込む

with open('./data/slu-sample-input.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

list_input_data = []
for line in lines:
    if not line.strip():
        continue
    list_input_data.append(line.strip())
    
    print(line.strip())

京都のおいしいラーメンを教えてください
今出川の近くでイタリアンはありますか
味亭の営業時間を教えて
割烹井上は何時から開いていますか
近くにおいしいそば屋はありますか
2000円以下で


## 意味文法による方法
まずは，意味文法を実装して、それに対して入力文をあてはめてみます。ここでは正規表現を使って実装します。
- 正規表現について下記が参考になります。
    - 参考：https://qiita.com/luohao0404/items/7135b2b96f9b0b196bf3

In [3]:
# 文法を定義する

# 要素の列挙
place = '(京都|今出川)'
genre = '(ラーメン|イタリアン|そば屋|そば)'
tellme = '(教えてください|教えて|教えてほしい)'
near = '(近く|辺り)'
there = '(ありますか|ありませんか)'
name = '(味亭|割烹井上)'

time_open = '営業時間'
time_from = '何時から'
time_until = '何時まで'
time_from_until = '(' + time_from + '|' + time_until + ')'

# 文法１（レストラン検索）
grammar1_1 = place + 'の(おいしい|美味しい)' + genre + 'を' + tellme
grammar1_2 = place + 'の' + near + 'で' + genre + 'は' + there

#
# 課題１　「近くにおいしいそば屋はありますか」がマッチするような文法を書いてみましょう
#
grammar1_3 = 'Nothing'

# 文法２（営業時間検索）
grammar2_1 = name + 'の' + time_open + 'を' + tellme
grammar2_2 = name + 'は' + time_from_until + '(ですか|開いていますか)'

# 文法３

# ユーザの意図と対応させる
grammars = [
    ['find', grammar1_1],
    ['find', grammar1_2],
    ['find', grammar1_3],   # 課題１用
    ['time', grammar2_1],
    ['time', grammar2_2]
]

for grammar in grammars:
    print(grammar[0] + ' : ' + grammar[1])

find : (京都|今出川)の(おいしい|美味しい)(ラーメン|イタリアン|そば屋|そば)を(教えてください|教えて|教えてほしい)
find : (京都|今出川)の(近く|辺り)で(ラーメン|イタリアン|そば屋|そば)は(ありますか|ありませんか)
find : Nothing
time : (味亭|割烹井上)の営業時間を(教えてください|教えて|教えてほしい)
time : (味亭|割烹井上)は(何時から|何時まで)(ですか|開いていますか)


In [4]:
# 入力文をマッチさせてみる
for data in list_input_data:
    
    print('入力：' + data)
    
    matched = False
    
    # 文法ごとにチェック
    for grammar in grammars:
        
        # 入力文が文法に完全マッチするかチェック
        result = re.match(grammar[1], data)
        
        # 文法にマッチした場合
        if result is not None:
            
            # 意図とマッチした文法を取得
            intent = grammar[0]
            grammer_matched = grammar[1]
            print('マッチ: ' + intent + ': ' + grammer_matched)
            matched = True
    
    if matched == False:
        print('マッチなし')
    
    print()

入力：京都のおいしいラーメンを教えてください
マッチ: find: (京都|今出川)の(おいしい|美味しい)(ラーメン|イタリアン|そば屋|そば)を(教えてください|教えて|教えてほしい)

入力：今出川の近くでイタリアンはありますか
マッチ: find: (京都|今出川)の(近く|辺り)で(ラーメン|イタリアン|そば屋|そば)は(ありますか|ありませんか)

入力：味亭の営業時間を教えて
マッチ: time: (味亭|割烹井上)の営業時間を(教えてください|教えて|教えてほしい)

入力：割烹井上は何時から開いていますか
マッチ: time: (味亭|割烹井上)は(何時から|何時まで)(ですか|開いていますか)

入力：近くにおいしいそば屋はありますか
マッチなし

入力：2000円以下で
マッチなし



In [5]:
# マッチしたら要素を抜き出す

# 抜き出す要素を定義
grammar_extract = {
    
    # 文法１
    'find': [
        ['place', place],
        ['genre', genre],
    ],
    
    # 文法２
    'time': [
        ['name', name],
        ['open', time_open],
        ['from', time_from],
        ['until', time_until],
    ]
}

for data in list_input_data:
    
    print('入力：' + data)
    
    matched = False
    
    # 文法ごとにチェック
    for grammar in grammars:
        
        # 入力文が文法に完全マッチするかチェック
        result = re.match(grammar[1], data)
        
        # 文法にマッチした場合
        if result is not None:
            
            matched = True
            
            # 文法にマッチしたら要素を抜き出す
            intent = grammar[0]
            for extract_pattern in grammar_extract[intent]:
                
                slot_name = extract_pattern[0]
                rule = extract_pattern[1]
                
                # 要素抽出を試みる
                result = re.findall(extract_pattern[1], data)
                if len(result) >= 1:
                    slot_value = result[0]
                
                    # 意図名、スロット名、スロット値を表示
                    print(intent + ' : ' + slot_name + ' : ' + slot_value)
            
            
    
    if matched == False:
        print('マッチなし')
    
    print()

入力：京都のおいしいラーメンを教えてください
find : place : 京都
find : genre : ラーメン

入力：今出川の近くでイタリアンはありますか
find : place : 今出川
find : genre : イタリアン

入力：味亭の営業時間を教えて
time : name : 味亭
time : open : 営業時間

入力：割烹井上は何時から開いていますか
time : name : 割烹井上
time : from : 何時から

入力：近くにおいしいそば屋はありますか
マッチなし

入力：2000円以下で
マッチなし



## 意味・格フレームによる方法
文法が発話の統語構造の一部を規定するのに対して、全く規定しない方法として意味・格フレームによる方法があります。ここでも正規表現を使って実装します。また、こちらの例ではユーザの意図は考慮せずにフレームのマッチングを行います。

In [6]:
# 意味フレームの列挙
place = '(京都|今出川)'
genre = '(ラーメン|イタリアン|そば)'
name = '(味亭|割烹井上)'
time_open = '営業時間'
time_from = '何時から'
time_until = '何時まで'

# 課題２　意味フレーム「2000円以下」などの予算を抽出する要素を追加しましょう
budget = 'Nothing'

# 各フレームの意味と対応させる
# ここにユーザ意図の種類を記述しておくことも可能
frames = [
    ['place', place],
    ['genre', genre],
    ['name', name],
    ['time_open', time_open],
    ['time_from', time_from],
    ['time_until', time_until],
    ['budget', budget]  # 課題２用
]

for frame in frames:
    print(frame[0] + ' : ' + frame[1])

place : (京都|今出川)
genre : (ラーメン|イタリアン|そば)
name : (味亭|割烹井上)
time_open : 営業時間
time_from : 何時から
time_until : 何時まで
budget : Nothing


In [7]:
# 入力文をマッチさせてみる
for data in list_input_data:
    
    print('入力：' + data)
    
    matched = False
    
    # フレームごとにチェック
    for frame in frames:
        
        # フレームが入力文に部分マッチするかチェック
        result = re.search(frame[1], data)
        
        if result is not None:
            meaning = frame[0]
            frame_matched = frame[1]
            
            # マッチした要素（フレーム値）も取り出す
            slot_value = result.group()
            
            # フレーム名，フレーム値を表示
            print(meaning + ' : ' + slot_value)
            
            matched = True
    
    if matched == False:
        print('マッチなし')
    
    print()

入力：京都のおいしいラーメンを教えてください
place : 京都
genre : ラーメン

入力：今出川の近くでイタリアンはありますか
place : 今出川
genre : イタリアン

入力：味亭の営業時間を教えて
name : 味亭
time_open : 営業時間

入力：割烹井上は何時から開いていますか
name : 割烹井上
time_from : 何時から

入力：近くにおいしいそば屋はありますか
genre : そば

入力：2000円以下で
マッチなし

